In [ ]:
# -q (quiet) 옵션은 설치 로그를 안보여줌(조용히함)
%pip install -q seaborn

In [ ]:
# 타이타닉(데이터 프레임 ) 데이터셋 불러오기
# (pandas에 기본 내장된 예제 데이터셋)
import seaborn as sns
import pandas as pd

# titanic 데이터 DataFrame
titanic = sns.load_dataset('titanic')

# 처음 5개 행 보기
print("타이타닉 데이터 미리보기:")
print(titanic.head())

# 데이터셋 정보 확인
print("\n데이터셋 정보:")
print(titanic.info())

# 데이터 요약 통계
print("\n데이터 요약 통계:")
print(titanic.describe())

In [ ]:
# Col 결측치 합계  
df = titanic
titanic.isnull().sum()
# 승객 등급 등장 빈도 (1, 2, 3 등석)
df['class'].value_counts()[0:3]
# 성별 빈도(남/녀)
df['sex'].value_counts()
# 생존 여부 빈도(생존자/사망자)
df['alive'].value_counts()

In [ ]:
#성별에 따른 생존율 survived 
survival_rate = df.groupby('sex')['survived'].mean()
#더 복잡한 톡예 가능 
titanic.groupby('sex').agg({'survived':['mean']})
#승객 등급에 따른 생존율 
survival_rate1 = df.groupby('pclass')['survived'].mean()
print(survival_rate, survival_rate1)
#전체 인원중 생존자 수 

In [ ]:
# 성별 & 승객 등급에 따른 생존
titanic.groupby(['sex', 'pclass'])['survived'].mean()

#피벗테이블 
titanic.pivot_table(
    values='survived',
    index='sex',
    columns='pclass',
    aggfunc='mean'
)
# 나이 그룹별 생존율 
titanic['age_group']= pd.cut(
    titanic['age'],
    bins=[0, 12, 18, 35, 60, 100,200], #0~12는 아동이고..
    labels=['아동', '청소년', '청년', '중장년', '노년','불사'] #초과~이하 

)
titanic.head(3)

#카테고리가 있는데 해당되는 내용이 없을 때/ observed 옵션
titanic.groupby('age_group',observed=True)['survived'].mean()
print(titanic.groupby('age_group',observed=True)['survived'].mean())

#성별 +나이그룹으로 생존율 확인


titanic.groupby(['sex', 'age_group'])['survived'].mean()

titanic.pivot_table(
    values='survived',
    index='age_group',
    columns='pclass',
    aggfunc='mean'
)
# 나이 그



In [ ]:
titanic = sns.load_dataset('titanic')
#결측치 확인 
missing = titanic.isna().sum()
#결측이 있는 항목만 확인 
missing[missing > 0]
#결측비율 
titanic.isna().mean() * 100


In [ ]:
# 결측치 채우기 
# age: 중요한 정보: 평균/ 중앙값 대체
#embarked : 가장 많은 사람들이 탄 곳으로 대체 edck: 
#deck 추측 불가능 (의미 없음) ->삭제 

#필요하다면, 카피떠서 진행 
titanic_processed =titanic.copy()

#나이 결측치 채우기 (남녀상관없이 전체 평균으로 채움)
mean_age = titanic['age'].mean()
titanic_processed['age'] = titanic['age'].fillna(mean_age)
#비어있던 행 마스킹
mask = titanic['age'].isna() 
titanic['age'].isna() .sum()

#평균으로 채워진 값들만 확인
titanic_processed.loc[mask]



In [ ]:
#성별/객실 별로 평균을 추정하는 건??

#모든 사람들을 성별과 객실 그룹의 평균나이로 바꾼 Series
mean_ages = titanic.groupby(['sex', 'pclass'])['age'].transform('mean')

#titanic 의 age col 중 빈 값만, mean_age 로 채움 
titanic_processed['age'] = titanic['age'].fillna(mean_ages)

titanic_processed.loc[titanic['age'].isna()]
#원래 비어있던 애들만 확인 



In [ ]:
#embarked (탑승도시) ->최빈값 (가장 많이 탄 곳)

#비어 있는 항구 개수 
titanic['embarked'].isna().sum()
#최빈값 (Series - 벡터)
mode_embarked = titanic['embarked'].mode()[0] #Series 에서 1개 뽑기 
#빈 값은 최빈값으로 교체 
titanic_processed['embarked'] = titanic['embarked'].fillna(mode_embarked)

#처리 완료 확인(빈값0개)
titanic_processed['embarked'].isna().sum()


In [ ]:
# dec =k 은 77% 가 비어있음-> 삭제 
titanic['deck'].isna().mean() * 100  #빈값 비율 
#'deck'컴럼은 삭제
titanic_processed.drop('deck',axis=1,inplace= True)
titanic_processed.info

In [75]:
titanic_processed.info

<bound method DataFrame.info of      survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_m